In [1]:
from matplotlib import pyplot as plt
import numpy as np
import importlib
from simulator.ideal import SimulatorIdeal
from simulator.lennard import SimulatorLennard
from simulator.magnetic import SimulatorMagnetic

import multiprocessing
from multiprocessing import Pool, cpu_count

# import logging.config
# import logging
from utils.logs import get_logger

import sys
# logging.config.fileConfig("/home/amatevosyan/telegramLogConfig")
# logger = logging.getLogger("telegram")

# logger = multiprocessing.log_to_stderr()
# logger.setLevel(logging.WARNING)

In [2]:
def before_step(obj, r, v, t):
    if not hasattr(obj, "Bz_dummy"):
        obj.Bz_dummy = obj.Bz
        obj.abc_dummy = obj.abc.copy()
        
        obj.Bz = 0
        obj.abc[1] = obj.abc[0]
        obj.init_potential_params()
        
    if t > 100:
        obj.Bz = obj.Bz_dummy
        obj.abc = obj.abc_dummy
        obj.init_potential_params()
        obj.last_a = None

In [3]:
pool = Pool(cpu_count(), maxtasksperchild=1); pool

<multiprocessing.pool.Pool state=RUN pool_size=12>

 12%|█▏        | 1165526/10000000 [07:26<1:17:56, 1889.12it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 21%|██        | 2088391/10000000 [16:11<1:10:13, 1877.85it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 30%|███       | 3006502/10000000 [24:39<1:20:47, 1442.56it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the con

In [4]:
params_model = {
    "group_name" : "BEnsemble 1",
    "R" : 1.0,
    "Rz" : 0.1,
    "eccentricity": 0.15,
    "sigma":0.5,
    "epsilon":1.0,
    "get_logger" : get_logger
}
params_init = {
    "energy": 1.0,
    "sigma_grid":0.55,
    "position_random_shift_percentage": 1.0/100,
    "planar": False,
    "zero_momentum": False,
}
params_simulation = {
    "iteration_time" : 1000,
    "dt" : 1e-4,
    "record_interval" : 1e-1,
    "algorithm" : "VERLET",
    "before_step" : before_step
}


In [5]:
sim1 = SimulatorMagnetic(Bz=1e-6, **params_model)
sim1.init_positions_velocities(**params_init)
sim1.rotational_push(0.8)
sim1.angular_momentum(sim1.r_init, sim1.v_init)[2].sum()

15.478481499435608

In [6]:
sim1.total_energy(sim1.r_init, sim1.v_init)

10.932360271703011

In [7]:
sim1.volume_fraction()

4.118045142242321

In [8]:
for i in range(10):
    name = f"subgroup-{i}"
    sim = SimulatorMagnetic(Bz=0, name=name, **params_model)
    sim.init_positions_velocities(**params_init)
    sim.rotational_push(0.8)
    r_init, v_init = sim.r_init, sim.v_init
    
    pool.apply_async(sim.simulate_async, kwds=params_simulation)
    
    for Bz in np.linspace(0.5*1e-6, 1.5*1e-6, 100):
        sim = SimulatorMagnetic(Bz=Bz, name=name, **params_model)
        sim.r_init, sim.v_init = r_init, v_init
        pool.apply_async(sim.simulate_async, kwds=params_simulation)

  2%|▏         | 189376/10000000 [01:00<54:26, 3003.72it/s]s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 16%|█▋        | 1629044/10000000 [12:01<1:14:22, 1875.96it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 33%|███▎      | 3300001/10000000 [27:25<1:03:18, 1764.08it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the conf